In [1]:
import requests
import pandas as pd

In [2]:
#url = f'https://graph.instagram.com/me/media?fields=id,caption,media_type,media_url,timestamp&access_token=

In [3]:
response = requests.get(url)

In [4]:
if response.status_code == 200:
    data = response.json()
else:
    print("Error making the API request:", response.text)
    data = None

In [5]:
data

{'data': [{'id': '18049687408639342',
   'caption': 'It gets tougher everyday',
   'media_type': 'IMAGE',
   'media_url': 'https://scontent.cdninstagram.com/v/t51.29350-15/440914603_1148341009700329_8794870000605352566_n.jpg?_nc_cat=105&ccb=1-7&_nc_sid=18de74&_nc_ohc=5CeyvfsYTl0Q7kNvgFC79b1&_nc_ht=scontent.cdninstagram.com&edm=ANo9K5cEAAAA&oh=00_AfAJtyRvedo79sr_uVYsFl7Onjc42qva6ukmLhRqdh7pgA&oe=663C0B92',
   'timestamp': '2024-05-04T10:29:34+0000'}],
 'paging': {'cursors': {'before': 'QVFIUmwwb1FXR3NxeU5ZAUWptdDBFVlJQSU1PbVprMk9GTHMxSHdwZAk42OGNyRzhab3gwWFE1TEJvTUlmdTg5MVZArU3BWMm84MHByaFZA0cTcwR1hVa2h2M3lR',
   'after': 'QVFIUmwwb1FXR3NxeU5ZAUWptdDBFVlJQSU1PbVprMk9GTHMxSHdwZAk42OGNyRzhab3gwWFE1TEJvTUlmdTg5MVZArU3BWMm84MHByaFZA0cTcwR1hVa2h2M3lR'}}}

In [6]:
if data:
    df = pd.DataFrame(data['data'])  # Assuming 'data' is a dictionary containing 'data' key with a list of records
else:
    df = pd.DataFrame(columns=['id', 'caption','mediatype','media_url','timestamp'])


In [7]:
print(df)

                  id                   caption media_type  \
0  18049687408639342  It gets tougher everyday      IMAGE   

                                           media_url                 timestamp  
0  https://scontent.cdninstagram.com/v/t51.29350-...  2024-05-04T10:29:34+0000  


In [8]:
df.to_csv('instagram_data.csv', index=False)

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.models import Model

from keras.layers import Embedding, Bidirectional, LSTM, Dense, Input, Concatenate, Dot, Activation

In [10]:
from ocr import get_text

In [11]:
training_data = pd.read_csv('training_data.csv')

In [12]:
instagram_data = pd.read_csv('instagram_data.csv')


In [13]:
new_data = []
for index, row in df.iterrows():
    new_row = {'id': row['id'], 'timestamp': row['timestamp']}
    flag = 0
    if row['media_type'] == 'IMAGE':
        # Perform OCR on the image URL
        ocr_text = get_text(row['media_url'])
        if ocr_text:
            new_row['text'] = ocr_text
            flag = 1
        else:
        # Use caption as text
            new_row['text'] = row['caption']    
    else:
        # Use caption as text
        new_row['text'] = row['caption']
    new_data.append(new_row)
    if flag == 1:
        new_row_2 = {'id': row['id'], 'timestamp': row['timestamp'],'text':row['caption']}
        new_data.append(new_row_2)

# Create new DataFrame
new_df = pd.DataFrame(new_data)
print(new_df)

                  id                 timestamp  \
0  18049687408639342  2024-05-04T10:29:34+0000   
1  18049687408639342  2024-05-04T10:29:34+0000   

                                                text  
0  Everyday it gets harder.\n\nEveryday it's like...  
1                           It gets tougher everyday  


In [14]:
new_df.to_csv('test.csv')

In [15]:
new_df.dropna(subset=['text'], inplace=True)
X_train = training_data['caption'].astype(str)
y_train = training_data['depression_label']
X_test = new_df['text'].astype(str)

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [17]:
max_len = 100  # Define your maximum sequence length
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [18]:
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100)(input_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
attention = Dense(1, activation='tanh')(lstm_layer)
attention = Activation('sigmoid')(attention)
context = Dot(axes=1)([attention, lstm_layer])
output_layer = Dense(1, activation='sigmoid')(context)

model = Model(inputs=[input_layer], outputs=output_layer)

In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(X_train_pad, y_train, epochs=10, batch_size=64)

Epoch 1/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 11s 60ms/step - accuracy: 0.8015 - loss: 0.5594
Epoch 2/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.9669 - loss: 0.1028
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - accuracy: 0.9830 - loss: 0.0628
Epoch 4/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 65ms/step - accuracy: 0.9889 - loss: 0.0422
Epoch 5/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - accuracy: 0.9933 - loss: 0.0270
Epoch 6/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - accuracy: 0.9962 - loss: 0.0168
Epoch 7/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - accuracy: 0.9950 - loss: 0.0203
Epoch 8/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - accuracy: 0.9954 - loss: 0.0146
Epoch 9/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - accuracy: 0.9970 - loss: 0.0116
Epoch 10/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - accuracy: 0.9978 - loss: 0.0074


In [21]:
predicted_labels = model.predict(X_test_pad)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step


In [22]:
new_df['predicted_depression_label'] = predicted_labels.reshape(-1)


In [23]:
depressing_captions = new_df[new_df['predicted_depression_label'] > 0.9]['text']


In [24]:
for text in depressing_captions:
    print(text)


Everyday it gets harder.

Everyday it's like a nightmare
progressing on.

Everyday is another prayer
echohing from my bones
asking God
to take me now.

Everyday I close up more.

Everyday it gets more impossible.

Everyday I feel myself die some more.

Everyday | lay on my bed
wondering how
I'll leave this hell.

I've never felt so much pain.



In [25]:
new_df

,id,timestamp,text,predicted_depression_label
0,18049687408639342,2024-05-04T10:29:34+0000,Everyday it gets harder.\n\nEveryday it's like...,1.000000
1,18049687408639342,2024-05-04T10:29:34+0000,It gets tougher everyday,0.547465
